In [5]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from src.experiment import get_fallacy_df, save_fallacy_df, get_classification_prompt_template
from src.tuning import tuning_train_test_split, get_tuning_examples, save_tuning_examples, TuningSet
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

# Fine-tuning

In [2]:
e5_filename = 'data/fallacies_e5.csv'
df_fallacies_e5 = get_fallacy_df(e5_filename, only_incorrect=True)

df_fallacies_e5.head()

[2024-10-28 12:48:28] Created new fallacy identification dataframe.


,step,entity,fallacy,label,category,subcategory
0,Since John asked Maria if she used the last of...,tepas,Argument from Silence,1,informal,insufficiency
1,Since Alice asked if Bob knew what an 'ossia' ...,ossia,Argument from Silence,1,informal,insufficiency
2,Since Alice claims that the Hausdorff contents...,hausdorff contents,Argument from Silence,1,informal,insufficiency
3,"Since Tom, a seasoned tugboater, said that ice...",tugboaters,Argument from Silence,1,informal,insufficiency
4,Since John accuses Mary of being terrified of ...,beewolf,Argument from Silence,1,informal,insufficiency


In [3]:
tuning_train_test_split(df_fallacies_e5, n_train=4, n_validation=1)
df_fallacies_e5['tuning'].value_counts()


tuning
test          1160
train          928
validation     232
Name: count, dtype: int64

In [4]:
prompt_template = get_classification_prompt_template()
train_examples = get_tuning_examples(df_fallacies_e5, prompt_template, TuningSet.TRAIN)
validation_examples = get_tuning_examples(df_fallacies_e5, prompt_template, TuningSet.VALIDATION)

save_tuning_examples(train_examples, 'data/tuning-train.jsonl')
save_tuning_examples(validation_examples, 'data/tuning-validation.jsonl')

In [8]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Upload the training and validation files to the OpenAI API
train_file = client.files.create(file=open("data/tuning-train.jsonl", "rb"), purpose="fine-tune")
validation_file = client.files.create(file=open("data/tuning-validation.jsonl", "rb"), purpose="fine-tune")

In [13]:
print('Train file ID:', train_file.id)
print('Validation file ID:',validation_file.id)

Train file ID: file-7SBSFbEnOwrO5x7A31H1Xqmx
Validation file ID: file-8Oz10aKtDTKculml3awjF1gz


In [11]:
# Fine tune model
# With n_train=4 and n_validation=1, we get 5,633,052 trained tokens and cost is $10.90
client.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4o-mini-2024-07-18",
    seed=42,
    suffix="fallacy-detection-v1"
)

FineTuningJob(id='ftjob-sX7St7Bo2Ti5UznfUP0EF1Zu', created_at=1730121016, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-k2KKJWldFPKvRIwPH9qZcydN', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-7SBSFbEnOwrO5x7A31H1Xqmx', validation_file='file-8Oz10aKtDTKculml3awjF1gz', estimated_finish=None, integrations=[], user_provided_suffix='fallacy-detection-v1')

In [14]:
save_fallacy_df(df_fallacies_e5, 'data/fallacies_e5.csv')